In [ ]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate,ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

llm = OpenAI(model="gpt-3.5-turbo")
poem_chat = ChatOpenAI(temperature=1.0)
explain_chat = ChatOpenAI(temperature=0.1)

poem_template = ChatPromptTemplate.from_messages([
    ("system","너는 시인이야. 너는 스스로 {author} 작가라고 믿고 있어."),
    ("human","{subject}와 관련된 시를 써줘"),
])


In [23]:
explain_template = ChatPromptTemplate.from_messages([
    ("system","너는 국어선생님이야. 수능출제를 위해 이 시를 분석해서 설명해줘. 시의 주제나 시점, 기법 등 문제 출제에 도움이 될만한 부분을 캐치해줘. 다만 시를 너무 변형해서는 안되고, 캐치할 부분을 모르겠다면, 아는 부분만 답변해주면 돼."),
    ("human","{poem}"),
])

In [24]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self,text):
        items = text.strip().split(",")
        return list(map(str.strip,items))
    

In [25]:
poem_chain = poem_template | poem_chat 

explain_chain = explain_template | explain_chat

final_chain = {"poem":poem_chain} | explain_chain 

In [ ]:
final_chain.invoke({
    "author" : "한강",
    "subject" : "ECMASCRIPT",
})

AIMessage(content='이 시는 ECMAScript(JavaScript)에 대한 찬양과 그것이 프로그래밍 세계에서 어떤 역할을 하는지를 노래한 것으로 보입니다. 시인은 ECMAScript 코드의 달콤함과 끊임없는 노고를 강조하며, 변수, 함수, 객체, 배열이 마치 춤추고 노래하며 감각적인 프로그래밍의 세계를 만들어가는 모습을 묘사하고 있습니다.\n\n이 시는 프로그래밍 언어인 ECMAScript(JavaScript)의 매력을 표현하고 있으며, 프로그래밍이라는 분야에서도 예술적인 측면이 있다는 것을 강조하고 있습니다. 이 시를 통해 ECMAScript(JavaScript)가 얼마나 다양하고 창의적인 프로그래밍을 가능하게 하는지를 강조할 수 있습니다. 또한, 시인이 사용한 은유와 상징들을 통해 ECMAScript(JavaScript)의 특징을 설명하고, 이를 통해 프로그래밍에 대한 열정과 창의성을 표현할 수 있을 것입니다.')

In [ ]:

from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts import PromptTemplate,ChatPromptTemplate

examples = [
    {
        "request" : "영화 코코 정보 알려줘",
        "response" : """
            감독 : 리 언크리치,
            주요 출연진 : 안소니 곤살레스, 가엘 가르시아 베르날, 벤자민 브랫,
            예산 : 약 1억 7,500만 달러,
            흥행 수익 : 약 8억 9,600만 달러, 
            영화의 장르 : 애니메이션, 판타지, 음악, 드라마, 어드벤처, 
            간단한 시놉시스 : 미겔은 음악을 사랑하는 12살 소년이지만, 그의 가족은 여러 세대에 걸쳐 음악을 금기시하고 있습니다. 우연히 전설적인 음악가 에르네스토 드 라 크루즈와 연관된 가족의 비밀을 알게 된 미겔은 멕시코 전통 축제 **'망자의 날'**에 죽은 자들의 세상으로 모험을 떠납니다. 그곳에서 그는 사기꾼 헥터와 힘을 합쳐 자신의 가족 역사를 밝히고, 음악의 진정한 의미를 찾으려 합니다.
            이 영화는 가족, 전통, 용서를 주제로 감동적이고 따뜻한 이야기를 선사하며, 아름다운 비주얼과 음악으로 전 세계 관객들에게 큰 사랑을 받았습니다.
        """
    },
    {
        "request" : "영화 해바라기 정보 알려줘",
        "response" : """
            감독 : 강석범
            주요 출연진 : 김래원, 김해숙, 허이재, 이재룡
            예산 : 약 30억 원 (추정)
            흥행 수익 : 약 150만 관객 (대한민국)
            영화의 장르 : 드라마, 액션, 느와르
            간단한 시놉시스 :
            과거를 청산하고 새로운 삶을 살기 위해 노력하는 전직 깡패 태식은 출소 후 어머니를 만나고 작은 시골 마을에서 평화로운 삶을 꿈꿉니다. 그러나 그의 주변에는 과거의 그림자와 새로운 위협이 끊임없이 다가오며, 태식은 다시 한번 폭력과 갈등 속에서 싸울 수밖에 없는 상황에 놓이게 됩니다.
            이 영화는 가족애와 속죄, 그리고 인간 본성의 갈등을 진지하게 다루며 강렬한 감정과 액션으로 관객들에게 깊은 인상을 남겼습니다.
        """
    },
    {
        "request" : "영화 마션 정보 알려줘",
        "response" : """
            감독 : 리들리 스콧
            주요 출연진 : 맷 데이먼, 제시카 차스테인, 크리스틴 위그, 제프 대니얼스, 마이클 페냐, 케이트 마라
            예산 : 약 1억 800만 달러
            흥행 수익 : 약 6억 3,000만 달러 (전 세계)
            영화의 장르 : SF, 드라마, 어드벤처
            간단한 시놉시스 :
            화성 탐사 임무 중 갑작스러운 폭풍으로 탐사대가 철수하는 과정에서, 탐사대원 마크 와트니는 사고로 인해 홀로 화성에 남겨집니다. 와트니는 자신이 살아있음을 지구에 알리고 화성에서 생존하기 위해 과학적 지식과 창의력을 총동원해 농작물을 재배하며 버텨 나갑니다. 동시에 지구에서는 NASA와 동료들이 와트니를 구조하기 위한 작전을 펼칩니다.
            이 영화는 인간의 생존 본능, 과학의 가능성, 그리고 협력의 중요성을 강조하며 감동과 스릴을 동시에 선사합니다. 맷 데이먼의 열연과 화려한 비주얼, 실제 과학적 사실에 기반한 스토리로 큰 호평을 받았습니다.
        """
    },
]

example_prompt = PromptTemplate.from_template("human:{request}\nai:{response}")

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="human:영화 {movie} 정보 알려줘",
    input_variables=["movie"]
)

chain = prompt | explain_chat

chain.invoke({"movie":"아이언맨"})

AIMessage(content='ai:\n            감독 : 존 파브로\n            주요 출연진 : 로버트 다우니 주니어, 기네스 팰트로, 제프 브리지스, 테렌스 하워드\n            예산 : 약 1억 4,000만 달러\n            흥행 수익 : 약 5억 8,000만 달러 (전 세계)\n            영화의 장르 : 액션, SF, 슈퍼히어로\n            간단한 시놉시스 :\n            천재 발명가이자 무기 제조업체 CEO인 토니 스타크는 아프가니스탄에서 납치되어 자신의 무기로 만든 철갑수트를 이용해 탈출합니다. 이후 자신의 회사의 무기 생산을 중단하고, 아이언맨 수트를 개발하여 세계의 평화를 위해 히어로로 활약합니다. 그러나 자신의 기술이 악용되어 적과의 대결을 펼치게 되는데...\n            이 영화는 슈퍼히어로의 탄생과 성장, 용기와 책임감을 다룬 작품으로 매력적인 캐릭터와 화려한 액션으로 많은 팬을 확보하며 마블 시네마틱 유니버스의 시작을 알립니다.')